In [ ]:
import os
import sys
import ast
import pickle

glp = os.path.abspath("../code")
if not glp in sys.path: sys.path.append( glp )

from global_paths import global_paths

if not global_paths["COBS"] in sys.path: sys.path.append( global_paths["COBS"] )

In [ ]:
import cobs
import torch
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess

In [ ]:
from BuildingOccupancy import Person, Meeting, WeeklyMeeting, OneTimeMeeting, BuildingOccupancy
from DefaultBuildings import Building_5ZoneAirCooled
from Agents import agent_constructor
from CentralController import ddpg_episode_mc
import RLCritics
import StateUtilities as SU

In [ ]:
import visualization_helper as vh

In [ ]:
dirname1 = "../checkpoints/s112/20210429-1555/"
dirname2 = "../checkpoints/s112b/20210429-1629/"
dirname3 = "../checkpoints/s112c/20210429-1642/"

In [ ]:
dfs1 = vh.convert_sqlite_to_df(dirname1)

In [ ]:
dfs2 = vh.convert_sqlite_to_df(dirname2)

In [ ]:
dfs3 = vh.convert_sqlite_to_df(dirname3)

In [ ]:
alldfs = [dfs1, dfs2, dfs3]
fig_width = 15

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=len(alldfs), figsize=(fig_width,4), sharex=False)
for idx, dfs in enumerate(alldfs):
    vh.plot_lr_epsilon(dfs, axes[idx])

In [ ]:
_, axes = plt.subplots(nrows=3, ncols=len(alldfs), figsize=(fig_width,10), sharex=False)
for idx, dfs in enumerate(alldfs):
    vh.plot_eees_mean_per_episode(dfs, axes[:, idx])

In [ ]:
_, axes = plt.subplots(nrows=5, ncols=len(alldfs), figsize=(fig_width,8), sharex=True)
for idx, dfs in enumerate(alldfs):
    vh.plot_eeesea(dfs, axes[:, idx])

In [ ]:
_, axes = plt.subplots(nrows=4, ncols=len(alldfs), figsize=(fig_width,6))
for idx, dfs in enumerate(alldfs):
    vh.plot_eeesea_frobnorm(dfs, axes[:, idx])

In [ ]:
last_available_eval_episodes = []
for idx, dfs in enumerate(alldfs):
    last_eval_episode = dfs['sees'].loc[:, "episode"].unique()[-1]
    last_available_eval_episodes.append(last_eval_episode)
    print(f"Last available evaluation episode for dfs{idx}: {last_eval_episode:6}")

<h3>Details: Temperatur Setpoints for one week</h3>

In [ ]:
#selected_episodes = [13120 - 1, 14480 - 1, 14480 - 1]
#selected_weeks    = [datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 1)]

In [ ]:
if not "selected_episodes" in locals().keys():
    selected_episodes = last_available_eval_episodes.copy()
if not "selected_weeks" in locals().keys():
    # the start of the evaluation episode
    selected_weeks = [datetime.datetime.strptime(dfs['sees'].iloc[0]["datetime"], "%Y-%m-%d %H:%M:%S")
                      for dfs in alldfs]

subdfs = []
for idx, dfs in enumerate(alldfs):
    subdfs1 = vh.select_week_and_episode(dfs, selected_episodes[idx], selected_weeks[idx])
    subdfs.append( subdfs1 )

In [ ]:
_, axes = plt.subplots(nrows=5, ncols=len(alldfs), figsize=(fig_width,6), sharex=True)
for idx, dfs in enumerate(alldfs):
    vh.plot_sees(subdfs[idx], axes[:, idx])

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=len(alldfs), figsize=(fig_width,2))
for idx, dfs in enumerate(alldfs):
    vh.plot_eees_only_mstpc(subdfs[idx], axes[idx])

In [ ]:
_, axes = plt.subplots(nrows=3, ncols=len(alldfs), figsize=(fig_width,8))
for idx, dfs in enumerate(alldfs):
    vh.plot_eees_normal(subdfs[idx], axes[:, idx]) #['eees']['reward'].plot()

In [ ]:
_, axes = plt.subplots(nrows=4, ncols=len(alldfs), figsize=(fig_width,12), sharex=True)
for idx, dfs in enumerate(alldfs):
    vh.plot_seeser_all_rooms(subdfs[idx], axes[:, idx])
# for individual rooms use plot_seeser(subdfs, room_id, ax)

In [ ]:
subdfs_rooms  = []
subdfs_agents = []

for idx, sdfs in enumerate(subdfs):
    sdfs_rooms  = sdfs["seeser"].loc[:, "room"].unique()
    sdfs_agents = sdfs["seesea"].loc[:, "agent_nr"].unique()
    subdfs_rooms.append(sdfs_rooms)
    subdfs_agents.append(sdfs_agents)
    print(f"Available Room  IDs in subdfs{idx}: {sdfs_rooms}")
    print(f"Available Agent IDs in subdfs{idx}: {sdfs_agents}", "\n")

In [ ]:
max_n_agents = max(2, max([len(sdfs_agents) for sdfs_agents in subdfs_agents]))
_, axes = plt.subplots(nrows=max_n_agents, ncols=len(alldfs), figsize=(fig_width,2*max_n_agents))
for idx, sdfs in enumerate(subdfs):
    for idx2, roomid, agentid in zip(range(len(subdfs_rooms[idx])), subdfs_rooms[idx], subdfs_agents[idx]):
        vh.plot_room_temp_agent_setpoint(sdfs, roomid, roomid+1, agentid, axes[idx2, idx])

In [ ]:
nrows = max_n_agents
_, axes = plt.subplots(nrows=nrows, ncols=len(alldfs), figsize=(fig_width,2*nrows), sharex=True)
for a in axes.flatten():
    a.ticklabel_format(useOffset=False, style='plain')
for idx, sdfs in enumerate(subdfs):
    vh.plot_seesea(sdfs, axes[:, idx])

<h3>Details for a day</h3>

In [ ]:
selected_periods     = [datetime.datetime(2017, 1, 3), datetime.datetime(2017, 1, 3), datetime.datetime(2017, 1, 3)]
selected_periods_end = [datetime.datetime(2017, 1, 4), datetime.datetime(2017, 1, 4), datetime.datetime(2017, 1, 4)]

subdfs = []
for idx, dfs in enumerate(alldfs):
    subdfs1 = vh.select_week_and_episode(dfs, selected_episodes[idx],
                                         selected_periods[idx], selected_periods_end[idx])
    subdfs.append(subdfs1)

In [ ]:
_, axes = plt.subplots(nrows=3, ncols=len(alldfs), figsize=(fig_width,8))
for idx, dfs in enumerate(alldfs):
    vh.plot_eees_normal(subdfs[idx], axes[:, idx]) #['eees']['reward'].plot()

In [ ]:
max_n_agents = max(2, max([len(sdfs_agents) for sdfs_agents in subdfs_agents]))
_, axes = plt.subplots(nrows=max_n_agents, ncols=len(alldfs), figsize=(fig_width,2*max_n_agents))
for idx, sdfs in enumerate(subdfs):
    for idx2, roomid, agentid in zip(range(len(subdfs_rooms[idx])), subdfs_rooms[idx], subdfs_agents[idx]):
        vh.plot_room_temp_agent_setpoint(sdfs, roomid, roomid+1, agentid, axes[idx2, idx])

In [ ]:
nrows = max_n_agents
_, axes = plt.subplots(nrows=nrows, ncols=len(alldfs), figsize=(fig_width,2*nrows), sharex=True)
for a in axes.flatten():
    a.ticklabel_format(useOffset=False, style='plain')
for idx, sdfs in enumerate(subdfs):
    vh.plot_seesea(sdfs, axes[:, idx])